# End-to-End Real-Time Bitcoin Pipeline Simulation

This notebook demonstrates a full simulation of the real-time Bitcoin ingestion pipeline built using the Azure SDK for Python.

We will:
1. Fetch live Bitcoin price using CoinGecko
2. Simulate streaming by wrapping the data into an event
3. Simulate receiving and uploading that event to Azure Blob Storage

All logic is powered by `bitcoin_utils.py`. This notebook simulates the actual pipeline in one place for demonstration.


### Imports and Setup

In [1]:
import os
import json
import time
import asyncio
from dotenv import load_dotenv
from azure.storage.blob.aio import BlobServiceClient

from bitcoin_utils import (
    fetch_bitcoin_price_usd,
    get_azure_async_credential,
    upload_buffer_to_blob
)


### Loading Environment Variables

In [2]:
# Load environment variables from .env
load_dotenv()


True

###  Fetch Real-Time Bitcoin Price

In [3]:
btc_price = fetch_bitcoin_price_usd()

if btc_price is not None:
    print(f"Live BTC price fetched: ${btc_price}")
else:
    raise ValueError("Failed to fetch Bitcoin price.")


Live BTC price fetched: $103711.0


### Simulate EventHub Stream Message

In [4]:
# Simulate streaming an event
event = {
    "currency": "BTC",
    "price_usd": btc_price,
    "timestamp": time.time()
}

event_batch = [event]  # Simulated buffer of one event

print("Simulated event prepared for 'streaming':")
print(json.dumps(event, indent=2))


Simulated event prepared for 'streaming':
{
  "currency": "BTC",
  "price_usd": 103711.0,
  "timestamp": 1747406808.3219166
}


### Simulate Receiving + Uploading Event to Blob

In [5]:
# Get Azure Blob Storage details from environment
container_name = os.getenv("STORAGE_CONTAINER_NAME")
account_url = os.getenv("STORAGE_ACCOUNT_URL")

assert container_name and account_url, "Missing container or account URL."

# Use async credential to connect and upload
async def simulate_receiver_upload():
    credential = get_azure_async_credential()
    blob_service = BlobServiceClient(account_url=account_url, credential=credential)
    await upload_buffer_to_blob(blob_service, container_name, event_batch)

await simulate_receiver_upload()


Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x00000201A56FE4B0>
Unclosed connector
connections: ['deque([(<aiohttp.client_proto.ResponseHandler object at 0x00000201A56E2F30>, 6522.046)])']
connector: <aiohttp.connector.TCPConnector object at 0x00000201A56FE600>
Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x00000201A4FC6840>
Unclosed connector
connections: ['deque([(<aiohttp.client_proto.ResponseHandler object at 0x00000201A56E3590>, 6522.265)])']
connector: <aiohttp.connector.TCPConnector object at 0x00000201A554C2F0>


## Simulation Complete

simulated a complete real-time ingestion pipeline:

-  Fetched live Bitcoin data
-  Created a streaming-style JSON event
-  Uploaded the event to Azure Blob Storage using the same logic as the receiver

You can now check your Azure portal under your blob container to verify that a new JSON file has been saved.
